In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install aif360

In [6]:
# !pip install BlackBoxAuditing==0.1.54

In [37]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [39]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [40]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import RicciDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_model_params_for_mult_repair_levels

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_disparate_impact

## Define Input Variables

In [41]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_ricci'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'ricci_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [42]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [43]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [44]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '431019cf-ba79-4256-8478-509186d8b0b5',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  431019cf-ba79-4256-8478-509186d8b0b5


## Initialize custom objects

In [45]:
data_loader = RicciDataset()
data_loader.X_data.head()

,Oral,Written,Combine,Position,Race
0,89.52,95,92.808,Captain,White
1,80.00,95,89.000,Captain,White
2,82.38,87,85.152,Captain,White
3,88.57,76,81.028,Captain,White
4,76.19,84,80.876,Captain,White


In [46]:
data_loader.y_data.value_counts()

0    62
1    56
Name: Promoted, dtype: int64

In [47]:
data_loader.X_data.shape

(118, 5)

In [48]:
data_loader.X_data['Race'].value_counts()

White        68
Non-White    50
Name: Race, dtype: int64

## Run experiment iterations

### Experiment iteration 1

In [49]:
# tuned_params_filenames = ['tuning_results_Ricci_alpha_0.0_20230806__143119.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [50]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [51]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
#                                    with_tuning=False,
#                                    tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-08-06 10:45:35 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, '
                                 '0.8, 0.9, 1.0]',
 'model_init_seed': 100,
 'session_uuid': '431019cf-ba79-4256-8478-509186d8b0b5'}




2023-08-06 10:45:35 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([84, 85, 37, 45, 97, 26, 101, 112, 21, 33, 11, 46], dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([84, 85, 37, 45, 97, 26, 101, 112, 21, 33, 11, 46], dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

2023-08-06 10:56:34 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
INFO:root:Successfully tested classifiers by bootstrap
2023-08-06 10:56:35 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics
INFO:root:Successfully computed predict proba metrics



[RandomForestClassifier] Metrics matrix:


,Metric,overall,Race_priv,Race_dis,Model_Seed,Model_Name,Model_Params
0,Mean,0.632549,0.633838,0.630744,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
1,Std,0.082542,0.083720,0.080892,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
2,IQR,0.109888,0.111569,0.107534,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
3,Aleatoric_Uncertainty,0.520923,0.516485,0.527138,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
4,Overall_Uncertainty,0.561430,0.557825,0.566475,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
5,Statistical_Bias,0.187634,0.177422,0.201929,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
6,Jitter,0.099606,0.104889,0.092211,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
7,Per_Sample_Accuracy,0.903333,0.929286,0.867000,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
8,Label_Stability,0.860000,0.858571,0.862000,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
9,TPR,1.000000,1.000000,1.000000,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."


2023-08-06 10:56:35 experiment_interface.py INFO    : Experiment run was successful!
INFO:root:Experiment run was successful!


### Experiment iteration 2

In [40]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Diabetes_alpha_0.0_20230805__102158.csv',
    'tuning_results_Diabetes_alpha_0.1_20230805__103027.csv',
    'tuning_results_Diabetes_alpha_0.2_20230805__102953.csv',
    'tuning_results_Diabetes_alpha_0.3_20230805__103437.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [41]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-08-05 10:00:05 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3]',
 'model_init_seed': 200,
 'session_uuid': 'f88d9187-81c1-496e-a809-65185798edd6'}




2023-08-05 10:00:06 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([   45, 31369,  7833, 14675, 17571, 27012, 15348, 24086, 26152,
            16484, 16793, 36148, 35949, 41185,  3126,  5804, 19698, 24548,
             2101, 12216],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([   45, 31369,  7833, 14675, 17571, 27012, 15348, 24086, 26152,
            16484, 16793, 36148, 35949, 41185,  3126,  5804, 19698, 24548,
             2101, 12216],
           dtype='int64')


Multiple alphas:   0%|          | 0/4 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.3
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_diabetes/tuning_results_Diabetes_alpha_0.3_20230805__103437.csv
RandomForestClassifier:  {'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 70, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Diabetes_alpha_0.0_20230805__102158.csv',
    'tuning_results_Diabetes_alpha_0.1_20230805__103027.csv',
    'tuning_results_Diabetes_alpha_0.2_20230805__102953.csv',
    'tuning_results_Diabetes_alpha_0.3_20230805__103437.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Diabetes_alpha_0.0_20230805__102158.csv',
    'tuning_results_Diabetes_alpha_0.1_20230805__103027.csv',
    'tuning_results_Diabetes_alpha_0.2_20230805__102953.csv',
    'tuning_results_Diabetes_alpha_0.3_20230805__103437.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [46]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Diabetes_alpha_0.0_20230805__102158.csv',
    'tuning_results_Diabetes_alpha_0.1_20230805__103027.csv',
    'tuning_results_Diabetes_alpha_0.2_20230805__102953.csv',
    'tuning_results_Diabetes_alpha_0.3_20230805__103437.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Diabetes_alpha_0.0_20230805__102158.csv',
    'tuning_results_Diabetes_alpha_0.1_20230805__103027.csv',
    'tuning_results_Diabetes_alpha_0.2_20230805__102953.csv',
    'tuning_results_Diabetes_alpha_0.3_20230805__103437.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)